## Imports

In [92]:
import pandas as pd
import requests

file = open('secret.txt','r')
api_key = file.read()

## Get all unique schools (gotta conserve api calls)

In [93]:
dtype_specification = {'Building IRN': str, 'District IRN': str} 

public_school = pd.read_csv('output.csv', dtype = dtype_specification)
public_school.drop_duplicates(subset='Building IRN', inplace=True)
cols_to_drop = ["White", "Asian", "Black", "Hispanic", "AYP Met?", "Year", "Preformance Index"]
public_school.drop(columns=cols_to_drop, inplace=True)
public_school
public_school["long"] = [x for x in range(len(public_school))]
public_school["lat"] = [x for x in range(len(public_school))]
final_return = public_school.head(0)
pb_test = public_school.head(10)
pb_test

,Competition,District Name,District IRN,Building Name,Building IRN,Building Adress,School Type,long,lat
0,NaN,Cleveland Municipal,043786,Albert B Hart,000026,"3901 E 74th St Cleveland, OH 44105-3808",K-8,0,0
1,NaN,Ada Exempted Village,045187,Ada Elementary School,000059,"435 Grand Ave Ada, OH 45810-1013",K-6,1,1
2,NaN,Ada Exempted Village,045187,Ada High School,000067,"435 Grand Ave Ada, OH 45810-1013",7-12,2,2
3,NaN,Sandusky City,044743,Adams Junior High School,000083,"318 Columbus Ave Sandusky, OH 44870-2616",7,3,3
4,NaN,Hamilton City,044107,Adams Elementary School,000091,"450 S F St Hamilton, OH 45013-3245",K-6,4,4
5,NaN,Meigs Local,048520,Meigs Intermediate School,000105,"36871 State Route 124 Middleport, OH 45760-9717",3-5,5,5
6,NaN,Tri-Valley Local,048876,Adamsville Elementary School,000117,"7950 East St Adamsville, OH 43802-9024",K-6,6,6
7,NaN,Hamilton Local,046953,Hamilton Intermediate School,000118,"5132 Parsons Ave Lockbourne, OH 43137-9216",4-6,7,7
8,NaN,Oak Hill Union Local,047761,Oak Hill Middle/High School,000119,"5063 State Route 93 Oak Hill, OH 45656-9408",6-12,8,8
9,NaN,Oak Hill Union Local,047761,Oak Hill Elementary,000124,"401 Evans St Oak Hill, OH 45656-1131",K-5,9,9


## Get Coord Method

In [94]:


def get_coordinates(api_key, address):
    # Base URL for Google Maps Geocoding API
    url = "https://maps.googleapis.com/maps/api/geocode/json?"

    # Parameters for the API request
    parameters = {
        "address": address,
        "key": api_key
    }

    # Making the request
    response = requests.get(url, params=parameters)
    data = response.json()

    # Check if the request was successful
    if data['status'] == 'OK':
        # Extracting latitude and longitude
        latitude = data['results'][0]['geometry']['location']['lat']
        longitude = data['results'][0]['geometry']['location']['lng']
        return latitude, longitude
    else:
        print("Error:", data['status'])
        return None, None


def setLongLat(row):

    print(row[4])
    latitude, longitude = get_coordinates(api_key, row["Building Adress"])
    new_row = ({
        'District Name' : row[1],
        'District IRN' : row[2],
        'Building Name' : row[3],
        'Building IRN' : row[4],
        'Building Adress' : row[5],
        'School Type' : row[6],
        'long' : longitude,
        'lat' : latitude
    })
    final_return.loc[len(final_return)] = new_row

    

    
# # Address you want to get coordinates for
# address = "3901 E 74th St Cleveland, OH   44105-3808"

# # Get coordinates
# latitude, longitude = get_coordinates(api_key, address)
# if latitude and longitude:
#     print("Latitude:", latitude)
#     print("Longitude:", longitude)


## Testing on Mini Dataset

In [95]:
# pb_test.apply(setLongLat, axis=1)
final_return

,Competition,District Name,District IRN,Building Name,Building IRN,Building Adress,School Type,long,lat


## Full Run 

In [96]:
public_school.apply(setLongLat, axis=1)

000026
000059
000067
000083
000091
000105
000117
000118
000119
000124
000125
000130
000135
000137
000138
000139
000140
000141
000143
000162
000177
000187
000190
000195
000197
000208
000222
000224
000231
000232
000233
000234
000236
000241
000242
000251
000257
000265
000268
000270
000277
000282
000287
000288
000291
000296
000297
000298
000299
000300
000301
000302
000303
000304
000305
000306
000311
000315
000316
000319
000320
000321
000338
000342
000343
000345
000348
000349
000350
000351
000352
000356
000360
000362
000363
000364
000380
000383
000384
000385
000386
000387
000388
000392
000396
000398
000400
000402
000406
000417
000420
000426
000427
000429
000435
000436
000445
000448
000450
000451
000455
000463
000467
000470
000471
000472
000473
000474
000475
000481
000489
000491
000493
000497
000501
000505
000509
000510
000511
000513
000519
000520
000523
000525
000527
000534
000539
000541
000543
000546
000547
000553
000556
000557
000558
000559
000560
000562
000564
000574
000575
000576
000577

0        None
1        None
2        None
3        None
4        None
         ... 
28837    None
28838    None
28839    None
28840    None
28841    None
Length: 3890, dtype: object

In [97]:
final_return

,Competition,District Name,District IRN,Building Name,Building IRN,Building Adress,School Type,long,lat
0,NaN,Cleveland Municipal,043786,Albert B Hart,000026,"3901 E 74th St Cleveland, OH 44105-3808",K-8,-81.636359,41.452498
1,NaN,Ada Exempted Village,045187,Ada Elementary School,000059,"435 Grand Ave Ada, OH 45810-1013",K-6,-83.829483,40.774738
2,NaN,Ada Exempted Village,045187,Ada High School,000067,"435 Grand Ave Ada, OH 45810-1013",7-12,-83.829483,40.774738
3,NaN,Sandusky City,044743,Adams Junior High School,000083,"318 Columbus Ave Sandusky, OH 44870-2616",7,-82.709801,41.454126
4,NaN,Hamilton City,044107,Adams Elementary School,000091,"450 S F St Hamilton, OH 45013-3245",K-6,-84.575490,39.401909
...,...,...,...,...,...,...,...,...,...
3885,NaN,Monroe Local,139303,Monroe Junior High School,016889,"220 Yankee Rd Monroe, OH, 45050-1042",?,-84.388725,39.448956
3886,NaN,Beavercreek City,047241,Ferguson Hall Freshman School,016890,"2680 Dayton Xenia Rd Beavercreek, OH, 45434-6416",?,-84.033760,39.723352
3887,NaN,Warrensville Heights City,045005,Westwood Elementary School,016917,4500 Warrensville Center Rd Warrensville Heigh...,?,-81.536764,41.437612
3888,NaN,Groveport Madison Local,046979,Cruiser Academy,016939,"4400 Marketing Pl Ste A Groveport, OH, 43125-9347",?,-82.889801,39.882519


In [98]:
final_return.to_csv('public_school_long_lat.csv', index=False)